# Online Book Reader

Design the data structures for an online book reader system.

## Assumptions

- Each **User** reads some **Book**s.
- Each Book has any number of **Checkpoint**s where the User can stop and resume reading.
- A User might be reading multiple Books at the same time. `Book.state = {Unread, Reading, Read}`.

In [42]:
from collections import deque


class Author:
    def __init__(self, name: str):
        self.name = name
        # TODO: first_name, middle_name, last_name, and more data like nationality, dob


class Book:
    SPLIT_SEQ = " "

    def __init__(self, uid: int, title: str, raw_text: str,  authors: list[Author] = None):
        self.uid = uid
        self.title = title
        self.authors = authors if authors is not None else []
        self.raw_text = raw_text
        self.chunks = self.get_chunks()

    @property
    def author(self):
        return ", ".join(author.name for author in self.authors)

    def get_chunks(self):
        text_chunks = self.raw_text.split(self.SPLIT_SEQ)
        return dict(enumerate(text_chunks))

    def __repr__(self):
        return f"{self.title} ({len(self.chunks)} chunks)"

    def __iter__(self):
        yield from enumerate(self.chunks)

    def __len__(self):
        return len(self.chunks)


class User:
    def __init__(self, uid: int, name: str):
        self.uid = uid
        self.name = name
        self.books = deque()
        self.bookmarks = {}
        self.last_checkpoint = {}

    @property
    def reading(self) -> list[Book]:
        return [bm.book for bm in self.bookmarks.values() if bm.progress > 0]
    
    def notify(self, msg: str):
        print(f"{self.name}: {msg}")

    def add_book(self, book: Book):
        self.books.append(book)
        self.bookmarks[book.uid] = Bookmark(user=self, book=book, position=0)
        self.notify(f"That's exciting, a new book from {book.author}. Have a nice journey!")

    def stop_reading(self, book: Book, at_index: int):
        self.bookmarks[book.uid].update_position(at_index)
        if at_index < len(book):
            self.notify(f"Left {book.title} at position {at_index}.")
        else:
            self.notify(f"Contratulations! You finished {book.title} from {book.author}.")
            del self.bookmarks[book.uid]
            
    
class Bookmark:
    def __init__(self, user: User, book: Book, position: int = 0):
        self.user = user
        self.book = book
        self.position = position

    @property
    def progress(self):
        return self.position/(len(book) - 1)

    def update_position(self, position: int):
        self.position = position

    def __repr__(self):
        return f"Bm({self.book.title}, {self.position}, {self.progress:.0%})"

    
jlb = Author("J.L. Borges")
book = Book(1, "Fictions", raw_text="Once upon a time there was a labyrinth", authors=[jlb])
user = User(1, "Juan")

user.add_book(book)
user.stop_reading(book, at_index=3)
user.stop_reading(book, at_index=5)
user.stop_reading(book, at_index=8)
user.bookmarks
user.reading
user.books


Juan: That's exciting, a new book from J.L. Borges. Have a nice journey!
Juan: Left Fictions at position 3.
Juan: Left Fictions at position 5.
Juan: Contratulations! You finished Fictions from J.L. Borges.


deque([Fictions (8 chunks)])

[]